## LDAvis 를 이용한 NMF 결과 시각화

Bag of words model (x) 과 미리 학습한 NMF 모델, 그리고 x 를 변환한 y 를 로딩합니다. `x` 는 (n docs, n terms) 크기이고, `y` 는 (n docs, n components) 크기입니다. NMF 모델의 x 를 y 로 변환하는 mapper 인 `components` 도 가져옵니다.

In [1]:
import pickle
from lovit_textmining_dataset.navernews_10days import get_bow

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')

with open('./2016-10-20-nmf.pkl', 'rb') as f:
    nmf = pickle.load(f)
with open('./2016-10-20-nmf_y.pkl', 'rb') as f:
    y = pickle.load(f)

components = nmf.components_

print(x.shape)
print(y.shape)
print(components.shape)

(30091, 9774)
(30091, 100)
(100, 9774)


`components` 는 토픽의 역할을 하며, nonnegative elements 로 이뤄져 있습니다. 하지만 각 row 의 L1 norm 은 제각각 다릅니다. 이를 확률 분포로 만들기 위하여 L1 normalization 을 수행합니다.

`y` 는 각 문서에 포함된 components 의 양이 표현되어 있습니다. 이 역시 nonnegative elements 로 이뤄져 있으며, L1 norm 이 제각각 다르기 때문에 이를 확률 분포로 만들기 위하여 L1 normalization 을 수행합니다.

때로는 빈 벡터 (empty vector) 로 표현된 문서가 있기도 합니다. 이 경우에는 `y` 의 row 가 zero vector 입니다. 하지만 LDAvis 는 확률 행렬이 입력되었을 때, 각 row 의 합이 1 이 되어야 합니다. Zero vector 에 대해서는 모든 elements 의 값을 토픽 개수의 역수로 입력합니다. `probability_normalization` 함수는 이 역할을 합니다.

In [2]:
import numpy as np
from sklearn.preprocessing import normalize

def nmf_to_topic_model(components, y):
    doc_topic_prob = normalize(y, norm='l1')
    topic_term_prob = normalize(components, norm='l1')
    return topic_term_prob, doc_topic_prob

def sum_bow(bow):
    doc_lengths = np.asarray(bow.sum(axis=1)).reshape(-1)
    term_frequency = np.asarray(bow.sum(axis=0)).reshape(-1)
    return doc_lengths, term_frequency

def probablity_normalization(prob):
    row_sum = prob.sum(axis=1)
    base = 1 / prob.shape[1]
    prob[np.where(row_sum == 0)[0]] = base
    return prob

`components` 와 `y` 를 각각 `topic_term_prob` 와 `doc_topic_prob` 로 변환합니다. 두 확률 행렬 모두 empty row 를 수정하기 위하여 `probability_normalization` 함수를 거칩니다.

Bag of words model 의 row sum 을 계산하면 데이터 전체에서의 단어 빈도수를 계산할 수 있고, column sum 을 계산하면 문서에 등장한 단어 개수 (문서 길이)를 계산할 수 있습니다. `sum_bow` 함수를 이용하여 이를 계산합니다.

In [3]:
topic_term_prob, doc_topic_prob = nmf_to_topic_model(components, y)
topic_term_prob = probablity_normalization(topic_term_prob)
doc_topic_prob = probablity_normalization(doc_topic_prob)

doc_lengths, term_frequency = sum_bow(x)

LDAvis 는 다섯개의 정보를 반드시 입력해야 합니다. 우리는 앞선 과정에서 이를 모두 준비하였습니다.

- `topic_term_prob` : numpy.ndarray 형식의 (토픽, 단어) 확률 행렬
- `doc_topic_prob`  : numpy.ndarray 형식의 (문서, 토픽) 확률 행렬
- `doc_lengths`     : numpy.ndarray 형식의 문서의 단어 개수
- `idx_to_vocab`    : list of str 형식의 단어 인덱스
- `term_frequency`  : numpy.ndarray 형식의 단어 빈도 벡터

Argument `R` 은 LDAvis 의 오른쪽 화면에 출력되는 키워드의 개수입니다.

prepared data 의 크기가 클 경우에는 IPython Notebook 의 결과값에 그림이 출력되지 않기도 합니다. `save_html` 을 이용하여 HTML 파일로 저장한 뒤 이를 확인합니다.

In [4]:
from pyLDAvis import prepare, show, save_html

prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    idx_to_vocab,
    term_frequency,
    R = 30 # num of displayed terms
)

save_html(prepared_data, 'nmf_to_ldavis.html')

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
